# Profiling code with `cProfile` and `IPython`

`%prun`: Run a statement through the python code profiler.

In cell mode, the additional code lines are appended to the (possibly empty) statement in the first line. Cell mode allows you to easily profile multiline blocks without having to put them in a separate function.

The given statement (which doesn't require quote marks) is run via the python profiler in a manner similar to the `profile.fun()` function. Namespaces are internally managed to work correctly; `profile.run` cannot be used in IPython because it makes certain assumptions about namespaces which do not hold under IPython.

There are various options which you can find out by typing a question mark after function.

In [2]:
import numpy as np

In [3]:
def step(*shape):
    # Create a random n-vector with +1 or -1 values.
    return 2 * (np.random.random_sample(shape) < .5) - 1

We profile the cell, sort the report by "cumulative time", limit it to 10 lines, and save it to a file named "prun0".

In [7]:
%%prun -s cumulative -q -l 10 -T prun0
n = 10000
iterations = 50
x = np.cumsum(step(iterations, n), axis=0)
bins = np.arange(-30, 30, 1)
y = np.vstack([np.histogram(x[i, :], bins)[0] for i in range(iterations)])

 
*** Profile printout saved to text file 'prun0'. 


In [6]:
!cat 'prun0'

         2829 function calls (2527 primitive calls) in 0.009 seconds

   Ordered by: cumulative time
   List reduced from 58 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.009    0.009 {built-in method builtins.exec}
        1    0.000    0.000    0.009    0.009 <string>:1(<module>)
   354/52    0.000    0.000    0.006    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.005    0.005 <string>:5(<listcomp>)
       50    0.000    0.000    0.005    0.000 <__array_function__ internals>:2(histogram)
       50    0.001    0.000    0.005    0.000 histograms.py:680(histogram)
        1    0.003    0.003    0.003    0.003 <ipython-input-3-6d31778a15f0>:1(step)
       50    0.000    0.000    0.002    0.000 <__array_function__ internals>:2(sort)
       50    0.000    0.000    0.002    0.000 fromnumeric.py:826(sort)
       50    0.001    0.000    0.0

### Create a script then run `cProfile`
This generates a binary file but needs `pstats` to display.

In [18]:
%%writefile myscript.py
def step(*shape): 
    # Create a random n-vector with +1 or -1 values.
    return 2 * (np.random.random_sample(shape) < .5) - 1

n = 10000
iterations = 50
x = np.cumsum(step(iterations, n), axis=0)
bins = np.arange(-30, 30, 1)
y = np.vstack([np.histogram(x[i,:], bins)[0]
           for i in range(iterations)])  

Overwriting myscript.py


In [16]:
!python -m cProfile -o profresults myscript.py

In [ ]:
import pstats
p = pstats.Stats('profresults')
p.strip_dirs().sort_stats('cumulative').print_stats()

You can use **SnakeViz** to view profile dumps in a GUI program
https://jiffyclub.github.io/snakeviz/

In [20]:
%load_ext snakeviz

In [21]:
%snakeviz 'profresults'

 
*** Profile stats marshalled to file '/var/folders/34/ztb7n7l13p79rmyk373rt1b80000gn/T/tmp6i7d1b9i'. 
Embedding SnakeViz in this document...
